In [1]:
import pandas as pd
import numpy as np

In [2]:
CHUNKS = 2

white_frames = []
black_frames = []

for i in range(CHUNKS):
    white_frames.append(pd.read_csv(f'database/white_{i*100+1}-{(i+1)*100}.csv'))
    black_frames.append(pd.read_csv(f'database/black_{i*100+1}-{(i+1)*100}.csv'))

white_all_df = pd.concat(white_frames).dropna()
black_all_df = pd.concat(black_frames).dropna()

white_all_df.shape[0], black_all_df.shape[0]

(6182, 6303)

In [3]:
white_all_df

,a1P,b1P,c1P,d1P,e1P,f1P,g1P,h1P,a2P,b2P,...,b8k,c8k,d8k,e8k,f8k,g8k,h8k,k,q,score
0,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,True,46
1,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,True,83
2,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,True,77
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,True,152
4,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,True,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,True,134
3110,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,True,320
3111,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,True,999998
3112,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,True,174


In [4]:
import chess
import chess.engine
import chess.pgn

In [5]:
PIECES = [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]
PLAYERS = [chess.WHITE, chess.BLACK]

fix_names_func = {chess.WHITE: lambda x: x.upper(), chess.BLACK: lambda x: x.lower()}

In [6]:
def get_columns_names():
    columns = []
    for player in PLAYERS:
        for piece in PIECES:
            symbol = fix_names_func[player](chess.piece_symbol(piece))
            columns.extend(map(lambda square: f"{square}{symbol}", chess.SQUARE_NAMES))
        columns.append(fix_names_func[player]("K"))
        columns.append(fix_names_func[player]("Q"))
    return columns

In [7]:
columns = get_columns_names()

white_df = white_all_df.drop_duplicates(subset=columns)
black_df = black_all_df.drop_duplicates(subset=columns)

white_df.shape[0], black_df.shape[0]

(5840, 5865)

In [8]:
SCORE_THRESHOLD = 50
white_dataset = white_df[(white_df['score'] >= SCORE_THRESHOLD) | (white_df['score'] <= -SCORE_THRESHOLD)]
black_dataset = black_df[(black_df['score'] >= SCORE_THRESHOLD) | (black_df['score'] <= -SCORE_THRESHOLD)]

white_dataset.shape[0], black_dataset.shape[0]

(4600, 4485)

# Classification

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = white_dataset.drop(['score'], axis=1)
y = white_dataset['score'] >= 0
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.33, random_state=1)

In [13]:
X_trn, y_trn, X_tst, y_tst = map(torch.tensor, (X_trn, y_trn, X_tst, y_tst))

ValueError: could not determine the shape of object type 'DataFrame'

In [11]:
X_trn.shape

(3082, 772)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor


# Define logistic regression model

class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(772, 2)

    def forward(self, x):
        x = x.view(-1, 772)
        return self.linear(x)

# Initialize model and optimizer
model = LogisticRegression()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Train the model
num_epochs = 10
batch_size = 32

#train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for i in range((len(X_trn)-1)//batch_size + 1):
        # Forward pass
        X_trn_batch = X_trn[batch_size*i:batch_size*(i+1)]
        y_trn_batch = y_trn[batch_size*i:batch_size*(i+1)]
        outputs = model(X_trn_batch)
        loss = nn.functional.cross_entropy(outputs, y_trn_batch)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss every 100 batches
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(X_trn), loss.item()))


# Test the model
#test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

with torch.no_grad():
    correct = 0
    total = 0
    
    
    outputs = model(X_tst)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == y_tst).sum().item()

    print('Accuracy of the model on the test images: {:.2f}%'.format(100 * correct / total))

AttributeError: 'DataFrame' object has no attribute 'view'